# Partie 1: Classification de Documents

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

## CNN

### Chargement des Données

In [2]:
# Données d'entrainement
train_data_complete = pd.read_csv("../data/allocine_genres_train.csv", sep=",")
train_data = train_data_complete[["titre", "synopsis", "genre"]]

# Données de test/validation
test_data_complete = pd.read_csv("../data/allocine_genres_test.csv", sep=",")
trest_data = test_data_complete[["titre", "synopsis", "genre"]]

Lister les classes et leur associer un identifiant unique.

In [3]:
# Liste des genres
genre_name = sorted(train_data.genre.unique().flatten())
print("Genres:", genre_name)
print("Nombre d'exemplaires:", len(train_data))

# Identifiant unique par genre
genre_index = {genre_name[i]:i for i in range(len(genre_name))}
genre_index

Genres: ['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction']
Nombre d'exemplaires: 2875


{'biopic': 0,
 'comédie': 1,
 'documentaire': 2,
 'drame': 3,
 'historique': 4,
 'horreur': 5,
 'policier': 6,
 'romance': 7,
 'science fiction': 8}

Remplacer les genres par la valeur numérique associée.

In [4]:
train_data = train_data.replace({"genre": genre_index})
train_data.head()

,titre,synopsis,genre
0,Le Crime de l' Orient - Express,"En visite à Istanbul , le célèbre détective be...",6
1,12 hommes en colère,Un jeune homme d' origine modeste est accusé d...,3
2,Après moi le bonheur,"Lorsque Marie-Laure , mère de quatre jeunes en...",3
3,Les Lumières de la ville,Un vagabond s’ éprend d’ une belle et jeune ve...,7
4,Les Chemins de la dignité,"L' histoire vraie de Carl Brashear , premier A...",0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data[["titre", "synopsis"]],
                                                    train_data[["genre"]],
                                                    test_size=0.2,
                                                    random_state=12, # Random seed for shuffle
                                                    shuffle=True)

In [6]:
X_train.head()

,titre,synopsis
2447,Le Coeur des hommes,"Alex , Antoine , Jeff et Manu , quatre amis à ..."
2741,Sliver,"Dirigeante d' une maison d' édition , Carly No..."
2051,Grande-Synthe,"Crise migratoire , pollution industrielle , ch..."
474,Dallas Buyers Club,"1986 , Dallas , Texas , une histoire vraie . R..."
1037,Resident Evil : Chapitre Final,"Alice , seule survivante de la bataille de Was..."


In [7]:
y_train.head()

,genre
2447,7
2741,6
2051,2
474,0
1037,5


## Indexation du Vocabulaire

In [8]:
def get_vectorizer(documents, max_voc_size=8000, max_seq_length= 200, batch_size=64):
	vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_voc_size, 
									output_sequence_length=max_seq_length)
	# Création du jeu de données à partir de X_train et constitution de lots de 128 instances
	text_ds = tf.data.Dataset.from_tensor_slices(documents).batch(batch_size)
	# Création du vocabulaire à partir des données d'entrée
	vectorizer.adapt(text_ds)
	return vectorizer

nananinanaire## Chargement de Plongement de Mots Pré-entraînés